# Sizing Assumptions

## Airloop

In [58]:
# assumptions for nominal flow
m = 33
dT = 15
c = 1005
ro = 1.17
V = m/ro
print(V)

V_cfm = V/0.00047194745
print(V_cfm)

28.205128205128208
59763.28128296531


In [59]:
V*200/0.7/V_cfm

0.13484212857142858

### Sources used

[has a downloadable excel tool](https://www.engineeringtoolbox.com/duct-friction-pressure-loss-d_444.html)

[examples by stulz.de](https://www.stulz.de/de/newsroom/blog/aer-eine-neue-kennzahl-fuer-die-effizienz-von-luftbewegung-in-rechenzentren-352/), 
using specific fan power slightly above example 1 in the assumptions below, scaled for our nominal flow as above.


### Resulting reasonable assumptions
coil 200 Pa

short pipe ((veloc^2 x ro)/2) + plenum : 100 Pa

note: plenum almost negligible


## Chilled water flow

In [60]:
Cn = 471000 #W
Cn_ton = Cn/3500
print(Cn_ton)

V_gpm = 1.2 * Cn_ton
V = V_gpm * 0.0000630901964
ro_w = 1000
m = V * ro_w
print(m)

134.57142857142858
10.188165430080002


### Evaporator flow

In [61]:
our_mn = 12 #kg/s
our_Vn_gpm = (12/1000)/0.0000630901964
print("Our model nominal evaporator flow in gpm/ton is:", our_Vn_gpm/Cn_ton)
print("Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of", 1.5)

Our model nominal evaporator flow in gpm/ton is: 1.4134046113429595
Source provides for a 130 ton chiller a nominal evaporator flow in gpm/ton of 1.5


#### Sources used

[source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/ADM-APN051-EN.pdf)

#### Resulting reasonable assumptions

Due to the similarity of the CHW flow an tonnage, we can use the evaporator nominal pressure drop from the source of 15kPa.

### Condenser water flow check

In [62]:
# 3 gpm is 0.14 kg/s, see Pauls method based on ASHRAE Standard 550 / 590
0.14*130

18.200000000000003

### Condenser, Evaporator, Coil, and WSE Pressure Drop
[based on examples towards the end of this taylor engineering article](https://www.taylor-engineering.com/wp-content/uploads/2020/04/ASHRAE_Journal_-_How_to_Design_Control_Waterside_Economizers.pdf), it can be assumed that the condenser water side pressure drop equals the pressure drop on the evaporator side. Same goes for WSE HX.

Assuming the cooling tower and the coil have the same pressure drop.

In [ ]:
20000

### Cooling Tower

Approach temperature [Source (PG&E, leaving tower water temperature minus ambient wet bulb
temperature](https://www.pge.com/includes/docs/pdfs/mybusiness/energysavingsrebates/incentivesbyindustry/DataCenters_BestPractices.pdf) recommends 5 - 7 F, 
conservatively assuming

In [47]:
approachT = 4 #K

### Sizing the tower fan

In [63]:
# cold water temperature (entering the condenser)
T_cond_in_max = 297 #K
T_wet_bulb_max = 296 #K

T_cond_in_max_C = 297 - 273 #K
T_wet_bulb_max_C = 296 - 273 #K

print("T_cond_in_max_C:", T_cond_in_max_C)
print("T_wet_bulb_max_C:", T_wet_bulb_max_C)

T_cond_in_max_C: 24
T_wet_bulb_max_C: 23


## Pauls method based on ASHRAE Standard 550 / 590

In [64]:
# ASHRAE Standard 550 / 590, [source](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/waterside-design/adm-apn045-en_0912.pdf) 
# Tower is sized at 3.0 GPM per ton of plant capacity.  Note that some designs reduce this to 2.0 or 2.5 GPM per ton. 
# Chiller efficiency is .57 KW / Ton.  Some machines are even more efficient - but it is hard to get much better 
# than .5 KW / ton at design conditions.
# Tower fan efficiency .046 KW / Ton
# Condenser water pump .054 KW / Ton
# Tower + Pump = .1 KW / Ton

# In terms of overall plant energy use it is valuable to think of the overall plant running around .6 - .8 KW / Ton.  
# Of that amount 85% is attributable to the chiller and roughly 15% to the tower fan and pump.  
# So in terms of the overall lift done by the plant 
# (difference between the CHW temperature and the outdoor air wet bulb) - the tower is roughly 
# five times more efficient than the chiller.  This is due mostly to the fact that the tower is evaporating water - rather 
# than trying to mechanically cool it.  So the more work we can have the tower due and the less the chiller needs to do the better.  
# This is why strategies like the use of WSE and CDW are so effective in reducing total plant efficiency.

In [66]:
# Tower fan efficiency .046 KW / Ton
print("Tower nominal fan power is thus: ", .046 * 142, "kW")

Tower nominal fan power is thus:  6.532 kW


## Piping

[source](https://www.advantageengineering.com/fyi/166/advantageFYI166.php)

6" CW at 2200 Pa/100ft

5" CHW Pa/100ft

For those with access, the compiled spreadsheet is [here](https://docs.google.com/spreadsheets/d/1XtaqeURHSJD2tsk9S-GBtri8GY4wZl62JM6LV3KOiVo/edit?usp=sharing)
For the others, please see the ChillerPlant package models. The piping resistances are for the most part passed as pressure drops in valves 